In [1]:
import os

from llama_index.vector_stores import AstraDBVectorStore
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    Document,
)
from llama_index.llama_dataset import download_llama_dataset
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv(find_dotenv())

True

In [3]:
from app.engine.loader import get_mock_documents

docs = get_mock_documents(["./data/frames/1.jpeg", "./data/frames/2.jpeg", "./data/frames/3.jpeg"])

In [9]:
from app.engine.context import create_service_context, create_storage_context

service_context = create_service_context()  
storage_context = create_storage_context()

In [10]:
index = VectorStoreIndex.from_documents(
    documents=docs,
    service_context=service_context,
    storage_context=storage_context,
)

In [7]:
engine = index.as_chat_engine(similarity_top_k=5, chat_mode="condense_plus_context")
engine.chat("What's the integration of llama index?")

AgentChatResponse(response='LlamaIndex offers easy integration with your outer application framework. This means it can', sources=[ToolOutput(content='type: transcript\nvideo_path: ./video/example.mp4\nstart_time: 0.0\nend_time: 10.0\ntext: Allows easy integrations with your outer application framework\n(e.g. with LangChain, Flask, Docker, ChatGPT, anything else).\nLlamaIndex provides tools for both beginner users and advanced users.\nOur high-level API allows beginner users to use LlamaIndex to ingest and\nquery their data in 5 lines of code. Our lower-level APIs allow advanced users to\ncustomize and extend any module (data connectors, indices, retrievers, query engines,\nreranking modules), to fit their needs.\n\nAllows easy integrations with your outer application framework\n(e.g. with LangChain, Flask, Docker, ChatGPT, anything else).\nLlamaIndex provides tools for both beginner users and advanced users.\nOur high-level API allows beginner users to use LlamaIndex to ingest and\nqu

In [ ]:
l = index.as_retriever().retrieve("What's the integration of llama index?")

In [ ]:
l